Pitch Predictor yay!

Step 1) get the goods

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
![ -d pitchpredictor ] || git clone https://github.com/pdex/pitch-predictor pitchpredictor
!(cd pitchpredictor; git pull --rebase)

Cloning into 'pitchpredictor'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 29 (delta 7), reused 14 (delta 1), pack-reused 0
Unpacking objects: 100% (29/29), done.
Already up to date.
Current branch master is up to date.


Step 2) get the data

In [1]:
!pip install -r pitchpredictor/requirements.txt

In [0]:
#@title Verify pip install {display-mode: "form"}
!pip freeze

absl-py==0.9.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
asgiref==3.2.7
astor==0.8.1
astropy==4.0.1.post1
astunparse==1.6.3
atari-py==0.2.6
atomicwrites==1.4.0
attrs==19.3.0
audioread==2.1.8
autograd==1.3
Babel==2.8.0
backcall==0.1.0
beautifulsoup4==4.6.3
bleach==3.1.5
blis==0.4.1
bokeh==1.4.0
boto==2.49.0
boto3==1.12.49
botocore==1.15.49
Bottleneck==1.3.2
branca==0.4.0
bs4==0.0.1
CacheControl==0.12.6
cachetools==3.1.1
catalogue==1.0.0
certifi==2020.4.5.1
cffi==1.14.0
chainer==6.5.0
chardet==3.0.4
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.4.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.2.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.5
cvxpy==1.0.31
cycler==0.10.0
cymem==2.0.3
Cython==0.29.17
daft==0.0.4
dask==2.12.0
dataclasses==0.7
datascience==0.10.6
decorator==4.4.2
defusedxml==0.6.0
descartes==1.1.0
dill==0.3.1.1
distributed==1.25.3
Django==3.0.5
dlib==19.18.0
docopt==0.6.2
docutils==0.15.2
dopami

In [0]:
import statsapi
import pickle
from random import shuffle
import os
from os.path import exists
from os import makedirs
from bz2 import open
from tqdm.notebook import tqdm
import dbm

def create_directories(base="./"):
  directories = ["data","models","ckpt"]
  for dir in directories:
    full = os.path.join(base, dir)
    if not exists(full):
      try:
        makedirs(full)
      except:
        print("ERROR: Could not create and/or access directory: {}".format(dir))
        raise

create_directories()
data_years = [2016,2017,2018,2019]

drive_dir = "drive/My Drive/pitch-predictor"
in_drive = lambda x: os.path.join(drive_dir, x)
schedule_filename = lambda y : "schedule-p4-" + str(y) + ".pkl.bz2"
game_filename = lambda x, y : "data/games-%d-id-%d-p4.pkl.bz2" % (x, y)
games_dir = lambda y : "data/games-%d" % y
schedules_dbm = in_drive("schedules.dbm")
games_dbm = lambda x: in_drive("games-%d" % x)
events_dbm = in_drive("events.dbm")

def fetch_schedule(year):
    filename = schedule_filename(year) 
    #print(year,": ",schedule_filename(year),games_filename(year))
    print(year, ": ", filename)
    if exists(filename):
        print("{} exists. Loading...".format(filename),end="")
        with open(filename,"rb") as infile:
            schedule = pickle.load(infile)
    else:
        print("{} does not exist. Downloading...".format(filename),end="")
        schedule = statsapi.schedule(start_date='01/01/'+str(year),end_date='12/31/'+str(year))
        with open(filename,"wb") as outfile:
            pickle.dump(schedule, outfile, protocol=4)
    print("DONE.")
    return schedule

def fetch_game(year, game_id):
    filename = game_filename(year, game_id) 
    if not exists(filename):
        #print("File (" + filename + ") found. No update.",flush=True)
        #else:
        #print("File (" + filename + ") NOT found. Downloading...",end="",flush=True)
        with open(filename, "wb") as outfile:
            game = statsapi.get("game",{"gamePk":game_id})
            pickle.dump(game, outfile, protocol=4)
        #print("DONE.")

def fetch_games(year, schedule):
    games_dir = os.path.join("data", "games-%d" % year)
    if not exists(games_dir):
        try:
            makedirs(games_dir)
        except:
            print("ERROR: Could not create and/or access directory: {}".format(dir))
            raise
    
    game_ids = sorted(set([x['game_id'] for x in schedule]))

    counter = 0
    for game_id in tqdm(game_ids):
        counter += 1
        #print("%4d/%4d " % (counter, len(game_ids)), end='')
        fetch_game(year, game_id)
    print("DONE.")

def fetch_year(year):
    schedule = fetch_schedule(year)
    fetch_games(year, schedule)

print("Loaded.")

Loaded.


In [0]:
fetch_year(2016)

2016 :  data/schedule-p4-2016.pkl.bz2
data/schedule-p4-2016.pkl.bz2 exists. Loading...DONE.



DONE.


In [0]:
fetch_year(2017)

2017 :  data/schedule-p4-2017.pkl.bz2
data/schedule-p4-2017.pkl.bz2 does not exist. Downloading...DONE.



DONE.


In [0]:
fetch_year(2018)

2018 :  data/schedule-p4-2018.pkl.bz2
data/schedule-p4-2018.pkl.bz2 does not exist. Downloading...DONE.



DONE.


In [0]:
fetch_year(2019)

2019 :  data/schedule-p4-2019.pkl.bz2
data/schedule-p4-2019.pkl.bz2 does not exist. Downloading...DONE.



DONE.


In [0]:
# populate dbm files
#games_dbm = lambda x: in_drive("games-%d.dbm" % x)
def convert_year(year):
  year_keys = []
  is_game_file = lambda x: x.startswith("id-")

  with dbm.open(games_dbm(year), "n") as db:
    data_dir = games_dir(year)
    if os.path.exists(data_dir):
      entries = os.listdir(data_dir)
      filtered = filter(is_game_file, tqdm(entries, total=len(entries)))
      for entry in filtered:
        with open(os.path.join(data_dir, entry), "rb") as infile:
          game = pickle.load(infile)
          game_id = game.get('gamePk')
          if game_id is not None:
            db['%s' % game_id] = game
          else:
            print("Invalid data for %s" % entry)

def convert_years(years):
  for year in years:
    convert_year(year)

try:
  convert_years(data_years)
except Exception as e:
  print(e)
  import traceback
  traceback.print_exc()

dbm mappings have byte or string elements only



Traceback (most recent call last):
  File "<ipython-input-58-430a93c48810>", line 24, in <module>
    convert_years(data_years)
  File "<ipython-input-58-430a93c48810>", line 21, in convert_years
    convert_year(year)
  File "<ipython-input-58-430a93c48810>", line 15, in convert_year
    db['%s' % game_id] = game
TypeError: dbm mappings have byte or string elements only


# load game years

This defines the functionality to load games from disk. The actual loading will happen further down.

In [0]:
# define event classes


class XPlayEvent:
  def __init__(self, max_pitches=None, start_time=None, play_result=None,
               inning=None, away_bat=None, away_score=None, home_score=None,
               pitcher_id=None, batter_id=None, men_on=None, pitcher_team=None,
               batter_team=None, pitcher=None, batter=None, batter_score=None,
               pitcher_score=None):
    self.max_pitches = max_pitches
    self.start_time = start_time
    self.play_result = play_result
    self.inning = inning
    self.away_bat = away_bat
    self.away_score = away_score
    self.home_score = home_score
    self.pitcher_id = pitcher_id
    self.batter_id = batter_id
    self.men_on = men_on
    self.pitcher_team = pitcher_team
    self.batter_team = batter_team
    self.pitcher = pitcher
    self.batter = batter
    self.batter_score = batter_score
    self.pitcher_score = pitcher_score


class XPitchEvent:
  def __init__(self, pe=None, balls=None, strikes=None, outs=None,
               pitch_num=None, pitch=None, prior_seq=None):
    self.pe = pe
    self.balls = balls
    self.strikes = strikes
    self.outs = outs
    self.pitch_num = pitch_num
    self.pitch = pitch
    self.prior_seq = prior_seq


# Category classes

class PlayResultCategory(object):
  FIELD_OUT = tuple([
    'balk',
    'batter_interference',
    'catcher_interf',
    'caught_stealing_2b',
    'caught_stealing_3b',
    'caught_stealing_home',
    'double_play',
    'fan_interference',
    'field_error',
    'field_out',
    'fielders_choice',
    'fielders_choice_out',
    'force_out',
    'game_advisory',
    'grounded_into_double_play',
    'hit_by_pitch',
    'other_out',
    'passed_ball',
    'pickoff_1b',
    'pickoff_2b',
    'pickoff_3b',
    'pickoff_caught_stealing_2b',
    'pickoff_caught_stealing_3b',
    'pickoff_caught_stealing_home',
    'pickoff_error_1b',
    'runner_double_play',
    'stolen_base_2b',
    'triple_play',
    'wild_pitch',
    'sac_bunt',
    'sac_bunt_double_play',
    'sac_fly',
    'sac_fly_double_play'
  ])
  WALK = tuple([
    'intent_walk',
    'walk'
  ])
  STRIKEOUT = tuple([
    'strikeout',
    'strikeout_double_play'
  ])
  HIT = tuple([
    'home_run',
    'single',
    'double',
    'triple'
  ])
  def __call__(self, play_result):
    if play_result in self.WALK:
      return 'walk'
    if play_result in self.STRIKEOUT:
      return 'strikeout'
    if play_result in self.HIT:
      return 'hit'
    return 'field_out'


# Parsing classes

class PlayerParser(object):
  '''Parse a game and return a dict of player_id to fullname'''
  def __call__(self, game):
    player_db = {}
    players = game['gameData']['players']
    for elt in players.values():
      player_db[elt['id']] = elt['fullName']
    return player_db


class TeamParser(object):
  '''Parse a game and return a dict of player_id to team_name'''
  SIDES = tuple([
    'home',
    'away'                 
  ])
  def __call__(self, game):
    team_db = {}
    for side in self.SIDES:
      team_name = game['liveData']['boxscore']['teams'][side]['team']['name']
      for player in game['liveData']['boxscore']['teams'][side]['players'].values():
        team_db[player['person']['id']] = team_name
    return team_db


class PlayEventParser(object):
  '''Parse a play and return a XPlayEvent'''
  def __init__(self):
    self.play_result_category = PlayResultCategory()

  def batter_score(self, away_bat, away_score, home_score):
    if away_bat:
      return away_score
    return home_score

  def pitcher_score(self, away_bat, away_score, home_score):
    if not away_bat:
      return away_score
    return home_score

  def __call__(self, team_db, player_db, play):
    away_bat = play['about']['isTopInning'] # bool
    away_score = play['result']['awayScore'] # num
    home_score = play['result']['homeScore'] # num
    pitcher_id = play['matchup']['pitcher']['id']
    batter_id = play['matchup']['batter']['id']

    return XPlayEvent(
      max_pitches=len(play['pitchIndex']),
      start_time=play['about'].get('startTime'),
      play_result=self.play_result_category(play['result']['eventType']), # string from vocab
      inning=play['about']['inning'], # num
      away_bat=away_bat,
      away_score=away_score,
      home_score=home_score,
      pitcher_id=pitcher_id,
      batter_id=batter_id,
      men_on=(0 if "Empty" == play['matchup']['splits']['menOnBase'] else 1),
      pitcher_team=team_db[pitcher_id],
      batter_team=team_db[batter_id],
      pitcher=player_db[pitcher_id],
      batter=player_db[batter_id],
      batter_score=self.batter_score(away_bat, away_score, home_score),
      pitcher_score=self.pitcher_score(away_bat, away_score, home_score),
    )

import copy

class PitchEvenParser(object):
  '''Parse a game and return a list of play events'''
  def __init__(self):
    self.play_event_parser = PlayEventParser()

  def filtered_pitches(self, play, play_event):
    is_pitch = (
      lambda i : play['playEvents'][i]["isPitch"]
      and "type" in play['playEvents'][i]['details']
    )
    return list(filter(is_pitch, range(play_event.max_pitches)))

  def parse(self, play, playevent, pitch_num, pitch_seq):
    return XPitchEvent(
      pe=playevent,
      balls=play['playEvents'][pitch_num]['count']['balls'],
      strikes=play['playEvents'][pitch_num]['count']['strikes'],
      outs=play['count']['outs'],
      pitch_num=pitch_num,
      pitch=play['playEvents'][pitch_num]['details']['type']['code'],
      prior_seq=pitch_seq,
    )

  def __call__(self, team_db, player_db, game):
    output = []
    number_of_plays = len(game['liveData']['plays']['allPlays'])
    for play_number in range(number_of_plays):
      play = game['liveData']['plays']['allPlays'][play_number]
      if ('event' in play['result']):
        play_event = self.play_event_parser(team_db, player_db, play)
        filtered_pitch_indices = self.filtered_pitches(play, play_event)
        pitch_seq = []
        for i in filtered_pitch_indices:
          pitch_event = self.parse(play, play_event, i, copy.deepcopy(pitch_seq))
          pitch_seq.append(pitch_event.pitch)
          output.append(pitch_event)
    return output




class GameParser(object):
  def __init__(self):
    self.team_parser = TeamParser()
    self.player_parser = PlayerParser()
    self.pitch_event_parser = PitchEvenParser()

  def __call__(self, game):
    team_db = self.team_parser(game)
    player_db = self.player_parser(game)
    return self.pitch_event_parser(team_db, player_db, game)



In [0]:
def events_filename(year):
  return "data/events-%d.pkl.bz2" % year

import os

def load_year(year, ignore_cache=False):
  parse = GameParser()
  is_game_file = lambda x: x.startswith("id-")
  filename = events_filename(year)
  events = []
  if os.path.exists(filename) and not ignore_cache:
    with open(filename, "rb") as infile:
      events = pickle.load(infile)
  else:
    data_dir = games_dir(year)
    if os.path.exists(data_dir):
      games = {}
      print("File (" + data_dir + ") found.",flush=True)
      entries = os.listdir(data_dir)
      filtered = filter(is_game_file, tqdm(entries, total=len(entries)))
      for entry in filtered:
        try:
          with open(os.path.join(data_dir, entry), "rb") as infile:
            game = pickle.load(infile)
            game_id = game.get('gamePk')
            if game_id is not None:
              games[game_id] = game
            else:
              print("Invalid data for %s" % entry)
        except:
          os.remove(os.path.join(data_dir, entry))
      for game in games.values():
        events.extend(parse(game))
      with open(filename, "wb") as outfile:
        pickle.dump(events, outfile, protocol=4)
      print("DONE")
    else:
      print("File (" + data_dir + ") NOT found.",flush=True)
  return events


def load_years(shuffled_years, testing, ignore_cache=False):
  valid_years = []
  all_events = []
  for year in shuffled_years:
    events = load_year(year)
    all_events.extend(events)
    valid_years.append(str(year))
    print("year(%d): %d events loaded" % (year, len(events)))
    if testing:
      break
  print("YEARS LOADED: {}".format(" ".join(valid_years)))
  return (valid_years, all_events)

In [0]:
TESTING_FLAG = False
import random
if TESTING_FLAG:
    print("WARNING: TESTING MODE! Set TESTING_FLAG = False to run full model.", flush=True)
    num_epochs = 1
else:
    num_epochs = 50 # You can increase this until loss plateaus.

all_games = {}
shuffled_years = list(data_years)
random.shuffle(shuffled_years)

valid_years, all_games = load_years(shuffled_years, TESTING_FLAG)

print('DONE.')

year(2016): 707355 events loaded
year(2017): 713821 events loaded
year(2018): 691778 events loaded
year(2019): 695296 events loaded
YEARS LOADED: 2016 2017 2018 2019
DONE.


In [0]:
from collections import Counter
from os.path import dirname, exists
from os import path, makedirs
from sklearn.model_selection import train_test_split
from time import strftime, localtime
import bz2
import copy
import numpy as np
import pickle
import random


from tqdm.keras import TqdmCallback

from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Activation, BatchNormalization
from keras.models import Sequential
from keras.optimizers import SGD
from keras.utils import to_categorical

start_time_str = strftime("%Y%m%d-%H%M",localtime())


def encode(data,vocab):
    encoded = to_categorical(vocab.index(data),num_classes=len(vocab))
    return encoded

def decode(datum,vocab):
    return vocab[np.argmax(datum)]

def get_playerDB_by_game(c_game):
    players = c_game['gameData']['players']
    player_db = {}
    for elt in players.values():
        player_db[elt['id']] = elt['fullName']
    return player_db

def get_teamDB_by_game(c_game):
    team_db = {}
    for side in ['home','away']:
        team_name = c_game['liveData']['boxscore']['teams'][side]['team']['name']
        for player in c_game['liveData']['boxscore']['teams'][side]['players'].values():
            team_db[player['person']['id']] = team_name
    return team_db



def arr2pe(pitch_array):
    raise NotImplementedError
    return False


def pitch_events_by_game(c_game):
    output = []
    team_db = get_teamDB_by_game(c_game)
    player_db = get_playerDB_by_game(c_game)
    number_of_plays = len(c_game['liveData']['plays']['allPlays'])
    for play_number in range(number_of_plays):
        play = c_game['liveData']['plays']['allPlays'][play_number]
        if ('event' in play['result']):
            play_event = XPlayEvent(play, team_db, player_db)
            filtered_pitch_indices = list(filter(lambda i : play['playEvents'][i]["isPitch"] and "type" in play['playEvents'][i]['details'], range(play_event.max_pitches)))
            pitch_seq = []
            for i in filtered_pitch_indices:
                pitch_event = XPitchEvent(play,play_event,i,copy.deepcopy(pitch_seq))
                pitch_seq.append(pitch_event.pitch)
                output.append(pitch_event)
    return output

def build_event_list():
  print("Making xevent_list...",end="",flush=True)
  xevent_list = []
  success = [xevent_list.extend(pitch_events_by_game(t_game)) for t_game in all_games.values()]
  xplay_list = {x.pe.start_time: x.pe for x in xevent_list}
  xplay_list = list(xplay_list.values())
  print("DONE.",flush=True)

def do_other_things():

  all_players = set(map(lambda x: x.pe.pitcher, xevent_list))
  all_players.update(map(lambda x: x.pe.batter, xevent_list))
  all_players = sorted(all_players)
  print("ALL PLAYERS[",len(all_players),"] ( e.g., ",random.sample(all_players,3),")")

  all_pitchtypes = sorted(set(map(lambda x: x.pitch, xevent_list)))
  print("ALL PITCHTYPES[",len(all_pitchtypes),"]:", all_pitchtypes)

  all_outcometypes = sorted(set(map(lambda x: x.pe.play_result, xevent_list)))
  print("ALL RESULTS[",len(all_outcometypes),"]:", all_outcometypes)


  print("Vectorizing players...",flush=True)
  all_player_pitchcounts = {player: [] for player in all_players}
  all_player_outcomecounts = {player: [] for player in all_players}
  for elt in xevent_list:
      all_player_pitchcounts[elt.pe.pitcher].append(elt.pitch)
      all_player_pitchcounts[elt.pe.batter].append(elt.pitch)
  for elt in xplay_list:
      all_player_outcomecounts[elt.pitcher].append(elt.play_result)
      all_player_outcomecounts[elt.batter].append(elt.play_result)

  all_pitch_counters = {x: Counter(all_player_pitchcounts[x]) for x in all_player_pitchcounts.keys()}
  all_outcome_counters = {x: Counter(all_player_outcomecounts[x]) for x in all_player_outcomecounts.keys()}

  all_vectors = {}
  for player in all_pitch_counters:
      all_vectors[player] = [all_pitch_counters[player][pitchtype] for pitchtype in all_pitchtypes]
      all_vectors[player].extend([all_outcome_counters[player][outcometype] for outcometype in all_outcometypes])

  random_player = random.choice(all_players)
  print(",".join(all_pitchtypes + all_outcometypes))
  print(",".join(map(lambda x : "{:2d}".format(x), all_vectors[random_player])))
  print("DONE.",flush=True)

def pe2arr(pitch_event):
    if len(pitch_event.prior_seq) > 0:
        last_pitch_id = all_pitchtypes.index(pitch_event.prior_seq[-1])
    else:
        last_pitch_id = -1
    out = [pitch_event.balls, pitch_event.strikes,
            pitch_event.outs, pitch_event.pe.batter_score,
            pitch_event.pe.pitcher_score, pitch_event.pe.inning, last_pitch_id]
    out.extend(all_vectors[pitch_event.pe.batter])
    out.extend(all_vectors[pitch_event.pe.pitcher])
    return np.array(out)

def pe2result(pitch_event):
    return encode(pitch_event.pitch,all_pitchtypes)


def train_model():
  print(pe2arr(random.choice(xevent_list)),flush=True)

  print("Splitting for evaluation...",end="",flush=True)
  xevent_list, eval_xevent_list = train_test_split(xevent_list,test_size=0.1)
  print("DONE.",flush=True)


  print("Making input/output arrays...",end="",flush=True)
  input_array = np.array(list(map(lambda x : pe2arr(x),xevent_list))) 
  output_array = np.array(list(map(lambda x : pe2result(x),xevent_list)))
  print("DONE.",flush=True)

  print("Building model...",end="",flush=True)
  model = Sequential()
  model.add(Dense(1000, input_shape=input_array[0].shape))
  model.add(BatchNormalization())
  model.add(Dense(1000))
  model.add(Dense(1000))
  model.add(Activation('relu'))
  model.add(Dense(len(output_array[0]), activation='softmax'))
  optimizer = SGD()
  print("DONE.")
  print(model.summary())

  model.compile(loss='kullback_leibler_divergence', metrics=['accuracy'], optimizer=optimizer)

  checkpoint_path = "./ckpt/pp-" + start_time_str + "-{epoch:04d}.ckpt"
  checkpoint_dir = dirname(checkpoint_path)

  cp_callback = ModelCheckpoint(filepath=checkpoint_path)

  # this split is more for just basic sanity checking at checkpoints.
  X_train, X_test, y_train, y_test = train_test_split(input_array, output_array, test_size=0.05)

  model.fit(X_train, y_train , epochs=num_epochs, verbose=0,
              callbacks=[cp_callback, TqdmCallback(verbose=2)], validation_data=(X_test,y_test))
  model_path = "./models/model-" + start_time_str + ".h5" 
  model.save(model_path)
  loss, acc = model.evaluate(X_test,y_test, verbose=2)
  print("Model accuracy: {:5.2f}%".format(100*acc))

def evaluate_model():
  thresholds = [0.05,0.1,0.2]
  place_threshold = 3
  success_above_threshold = Counter()
  success_above_rank = 0
  exact_success = 0
  validation_sample_size = len(eval_xevent_list)

  print("Evaluating against {} samples...".format(validation_sample_size),end="",flush=True)
  for x in tqdm(eval_xevent_list):
      prediction = model.predict(np.array([pe2arr(x)]))
      prediction_list = prediction.tolist()[0]
      index_pitch = all_pitchtypes.index(x.pitch)
      index_predmax = np.argmax(prediction_list)
      nth_rank = sorted(prediction_list,reverse=True)[place_threshold]
      for threshold in thresholds:
          if (index_pitch in filter(lambda p : prediction_list[p] > threshold, range(len(prediction_list)))):
              success_above_threshold[threshold] += 1
      if (index_pitch in filter(lambda p : prediction_list[p] > nth_rank, range(len(prediction_list)))):
          success_above_rank += 1
      if (index_pitch == index_predmax):
          exact_success += 1
  print("DONE.")

  for threshold in thresholds:
      percent_above_threshold = (100.0*success_above_threshold[threshold])/validation_sample_size
      print("{:2.2f}% above {:2.0f}% predicted chance".format(percent_above_threshold,(100*threshold)))
  percent_above_rank = (100.0*success_above_rank)/validation_sample_size
  print("{:2.2f}% at rank {} or better".format(percent_above_rank,place_threshold))
  percent_exact_correct = (100.0*exact_success)/validation_sample_size
  print("{:2.2f}% exact success".format(percent_exact_correct,place_threshold))


Using TensorFlow backend.
